# Task: Create the MSAVI-2 Index using Sentinel-2/Landsat-8 using given input of 3 locations and dates 


Requirements: 

1. Install Anaconda and Jupyter Notebook through it.
2. Create an environment in anaconda to install python libraries and packages in it.
3. Install Python libraries and packages using anaconda prompt to the created environment address.

In [ ]:
'''Here, we have to import necessary python library n package provided from Google Earth Engine and then activate google earth engine 
authentication with using your personal credentials'''

#Import Python library and package especially to add Google Earth Engine functionalities into the map 
import ee
import geemap

When asked for credentials, put your google api key for successful authentication.
(Create account on 'Google Cloud Platform' to get your personal api key)

In [ ]:
#Add a google map using geemap package
Map = geemap.Map()

#Upload the landsat-8 satellite image directly from google earth engine datasets and add to the map
sat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1")
Map.addLayer(sat_img)
Map

# Add Geometries to the map

In [ ]:
#Set variables to add the geometries to the map using given coordinates for the 3 given locations
poly1 = ee.Geometry.Polygon([[

  [75.45157492160797,30.634404129904425],
  [75.4524278640747,30.63372099804389],
  [75.45236885547638,30.634399514164798],
  [75.45157492160797,30.634404129904425]

  ]] )

poly2 = ee.Geometry.Polygon([[

[85.85622847080231,26.09425078918021],
[85.85590660572052,26.093581136401006],
[85.85663080215454,26.09337879451938 ],
[85.85686147212981,26.094009907326967],
[85.85622847080231,26.09425078918021]

  ]] )

poly3 = ee.Geometry.Polygon([[

[78.66571158170699,17.66869116558751],
[78.6662346124649,17.6686911655875],
[78.6662346124649,17.66929686130703],
[78.66571158170699,17.66929686130703],
[78.66571158170699,17.66869116558751]

  ]] )

Now, we have to set the zoom level to add the geometry layers at a certain extent according to the map scale

In [ ]:
#set the zoom level of the map to a certain extent
Map.centerObject(poly1,30)
Map.addLayer(poly1)

Map.centerObject(poly2,30)
Map.addLayer(poly2)

Map.centerObject(poly3,30)
Map.addLayer(poly3)

Before proceeding to the image processing, we have to remove clouds from the satellite image to get accurate results further.

In [ ]:
#set a function for cloud removal task
def maskClouds(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int() # # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudsBitMask = ee.Number(2).pow(5).int()
    
    # # Get the pixel QA band.
    qa = image.select('pixel_qa')
    
    # # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    
    # # Return the masked image, scaled to [0, 1].    
    return image.updateMask(mask).divide(10000).copyProperties(image, ["system:time_start"])

# MSAVI2 Time Series Analysis

 
   **(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2**
   
   Using the above expression, we will calculate MSAVI2 for the LANDSAT-8 image and for that we have to set a function as following:

In [ ]:
#set a function with formula to calculate MSAVI using specific bands
def addMSAVI2(image):
    msavi = image.expression(
    '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2',
    {
      'NIR': image.select('B5'),
      'RED': image.select('B4')
    }
    ).rename('MSAVI')
    return image.addBands(msavi)

Let's move to the time series chart making process.

In [ ]:
#set the dates and the conditions to visualise the chart results for 3 specific locations
startDate ='2020-01-01'
endDate ='2020-06-30'

msavi_poly1 = sat_img.filterDate(startDate,endDate) \
    .filterBounds(poly1) \
    .map(maskClouds) \
    .map(addMSAVI2)

print(msavi_poly1)
# Map.addLayer(msavi_poly1)

msavi2_2 = sat_img.filterDate(startDate,endDate) \
    .filterBounds(poly2) \
    .map(maskClouds) \
    .map(addMSAVI2)

print(msavi_poly2)
# Map.addLayer(msavi_poly2)

msavi_poly3 = sat_img.filterDate(startDate,endDate) \
    .filterBounds(poly3) \
    .map(maskClouds) \
    .map(addMSAVI2)

print(msavi_poly3)
# Map.addLayer(msavi_poly3)

# Time Series Chart for 1st location

In [ ]:
#prepare a chart for 1st place/location

place1 = ui.Chart.image.seriesByRegion({
    'imageCollection': msavi2_2,
    'band': 'MSAVI',
    'regions': poly1,
    'reducer': ee.Reducer.mean(),
    'scale': 30,
    'seriesProperty': 'label',
    'xProperty': 'system:time_start'}).setChartType('LineChart').setOptions({
    'title': 'MSAVI2 Time Series Map for poly1 ',
    'hAxis': {'title': 'Date', 'titleTextStyle': {'italic': False, 'bold': True}},
    'vAxis': {'title': 'MSAVI2 Value ranges', 'titleTextStyle': {'italic': False, 'bold': True}}
    'lineWidth': 5,
    'colors': ['f0af07', '0f8755', '76b349']})
print(place1)

# Time Series Chart for 2nd location

In [ ]:
#prepare a chart for 2nd place/location
place_2 =
ui.Chart.image.seriesByRegion({
    'imageCollection': msavi2_2,
    'band': 'MSAVI',
    'regions': poly2,
    'reducer': ee.Reducer.mean(),
    'scale': 30,
    'seriesProperty': 'label',
    'xProperty': 'system:time_start'}).setChartType('LineChart').setOptions({
    'title': 'MSAVI2 Time Series Map for poly2 ',
    'hAxis': {'title': 'Date', 'titleTextStyle': {'italic': False, 'bold': True}},
    'vAxis': {'title': 'MSAVI2 Value ranges', 'titleTextStyle': {'italic': False, 'bold': True}}
    'lineWidth': 5,
    'colors': ['f0af07', '0f8755', '76b349']})
print(place_2)

# Time Series Chart for 3rd location

In [ ]:
#prepare a chart for 3rd place/location
place_3 =
ui.Chart.image.seriesByRegion({
    'imageCollection': msavi2_2,
    'band': 'MSAVI',
    'regions': poly3,
    'reducer': ee.Reducer.mean(),
    'scale': 30,
    'seriesProperty': 'label',
    'xProperty': 'system:time_start'}).setChartType('LineChart').setOptions({
    'title': 'MSAVI2 Time Series Map for poly3 ',
    'hAxis': {'title': 'Date', 'titleTextStyle': {'italic': False, 'bold': True}},
    'vAxis': {'title': 'MSAVI2 Value ranges', 'titleTextStyle': {'italic': False, 'bold': True}}
    'lineWidth': 5,
    'colors': ['f0af07', '0f8755', '76b349']})
print(place_3)